# Visualise both lidar and radar

You can find the dataset here: https://www.astyx.com/development/astyx-hires2019-dataset.html

In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import json
import IPython
import json
import base64
import numpy as np

%matplotlib inline

folder_lidar = "/Users/rmeertens/Downloads/dataset_astyx_hires2019/dataset_astyx_demo/lidar_os1/"
folder_radar = "/Users/rmeertens/Downloads/dataset_astyx_hires2019/dataset_astyx_demo/radar_hires1/"
folder_bbox = "/Users/rmeertens/Downloads/dataset_astyx_hires2019/dataset_astyx_demo/groundtruth_obj3d/"

# Note that calibration is always the same for these few examples I use
with open("/Users/rmeertens/Downloads/dataset_astyx_hires2019/dataset_astyx_demo/calibration/000000.json") as json_file:
    data = json.load(json_file)
    for a in data['sensors']:
        if a['sensor_uid'] == 'lidar_os1':
            lidar_to_radar = a['calib_data']['T_to_ref_COS']


In [2]:
def quaternion_to_euler(x, y, z, w):
    """Copied from stackoverflow: https://stackoverflow.com/questions/53033620/how-to-convert-euler-angles-to-quaternions-and-get-the-same-euler-angles-back-fr?rq=1
    
    Returns the angles in degrees (not radians)"""
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    X = math.degrees(math.atan2(t0, t1))

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.degrees(math.asin(t2))

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    Z = math.degrees(math.atan2(t3, t4))

    return X, Y, Z 

In [3]:
def read_radar_points(filename):
    # Read the CSV
    with open(filename) as txt: 
        lines = [line for line in txt]
    lines = lines[2:]

    # Go through all radar points and give them an RGB color
    xyz = list()
    for line in lines: 
        x, y, z, vr,mag = line.split(" ")
        vr = (float(vr)+5.0)/10.0
        r,g,b,alpha = plt.cm.brg(vr)
        xyz.append([x,y,z, r,g,b])
    radarpoints = np.array(xyz, dtype=np.float32)

    return radarpoints

def read_lidar_points(filename):
    # Read the CSV
    with open(filename) as txt: 
        lines = [line for line in txt]
    lines = lines[2:]

    # Go through all lidar points and give them an RGB color based on their reflectivity
    xyz = list()
    for line in lines: 
        X, Y, Z, Intensity, LaserID, T, Reflectivity, Noise, Range = line.split(" ")
        Reflectivity = np.clip(float(Reflectivity)/100.0, 0.0, 255.0)
        Reflectivity /= 255.0
        r,g,b,alpha = plt.cm.binary(Reflectivity)
        
        xyz.append([X,Y,Z, r,g,b])
    lidarpoints = np.array(xyz, dtype=np.float32)
    return lidarpoints


In [4]:
for file in ["000000.txt", "000001.txt", "000002.txt", "000003.txt", "000004.txt"]:
    radarpoints = read_radar_points(folder_radar + file)
    lidarpoints = read_lidar_points(folder_lidar + file)
    
    # Transform the lidar points to the radar space
    # Note that we want to keep the correct colours... 
    backup = lidarpoints.copy()
    lidarpoints = lidarpoints[:,:4]
    lidarpoints[:,3] = 0.0
    lidarpoints = np.dot(lidarpoints, np.transpose(lidar_to_radar))
    backup[:,:3] = lidarpoints[:,:3]
    lidarpoints = backup

    # Combine all lidar and radar points in one pointcloud
    combined = np.vstack((lidarpoints, radarpoints))
    
    # Switch dimensions around for better visualization in Three.js
    combined = combined[:,[0,2,1,3,4,5]]

    # Rescale the pointcloud so we can easily walk through it
    combined[:,:3] *= 0.1

    # Save to file ready to upload
    combined.byteswap().tofile('../astyx'+file+'.xyzrgb')
    
    # Load the bounding boxes to add them to the visualisation. 
    with open(folder_bbox+file.split('.')[0]+'.json') as json_file:
        data = json.load(json_file)
    
    # Load the bounding box
    towrite = list()
    for objecttest in data['objects']:
        ob = list()
        x,y,z = objecttest['center3d']
        ob.extend([x,z,y]) # This is the position of the object
        x,y,z = objecttest['dimension3d']
        ob.extend([x,z,y]) # This is the width-lenght-height
        
        x,y,z = quaternion_to_euler(*objecttest['orientation_quat'])
        ob.extend([0.0,math.radians(x),0.0]) # This is the rotation. We only rotate around the y axis in vr
        ob.extend([0.9, 1.0, 0.5, 0.5]) # This is the colour 
        towrite.append(ob)
    bboxes = np.array(towrite, dtype=np.float32)
    bboxes[:,:6] *= 0.1 # Resize the locations and dimensions so you can walk through it. 

    bboxes.astype(np.float32).byteswap().tofile('../astyxbbox'+file+'.bbox')   

In [5]:
# Visualise one example to verify it all works. 
i = 0
todisplay = {
    "points": [
        {
            "source": "url",
            "url": "astyx00000"+str(i)+".txt.xyzrgb",
            "type": "XYZRGB",
        }
    ],
    "bboxes": [
        {     
            "source": "url",
            "url": "astyxbbox00000"+str(i)+".txt.bbox"
        }
    ]
}

data = json.dumps(todisplay)
data = base64.urlsafe_b64encode(json.dumps(todisplay).encode())

url = 'http://127.0.0.1:8000/oculus.html?jsonb64=' + data.decode('ascii')
print(url)
iframe = '<iframe src=' + url + ' width=1000 height=550></iframe>'
IPython.display.HTML(iframe)
    


http://127.0.0.1:8000/oculus.html?jsonb64=eyJwb2ludHMiOiBbeyJzb3VyY2UiOiAidXJsIiwgInVybCI6ICJhc3R5eDAwMDAwMC50eHQueHl6cmdiIiwgInR5cGUiOiAiWFlaUkdCIn1dLCAiYmJveGVzIjogW3sic291cmNlIjogInVybCIsICJ1cmwiOiAiYXN0eXhiYm94MDAwMDAwLnR4dC5iYm94In1dfQ==


In [6]:
# Generate multiple links using the data in my S3 public bucket. 
    
for i in range(5):
    todisplay = {
        "points": [
            {
                "source": "url",
                "url": "https://pointcloudvisualization.s3.eu-central-1.amazonaws.com/astyx/astyx00000"+str(i)+".txt.xyzrgb",
                "type": "XYZRGB",
            }
        ],
        "bboxes": [
            {     
                "source": "url",
                "url": "https://pointcloudvisualization.s3.eu-central-1.amazonaws.com/astyx/astyxbbox00000"+str(i)+".txt.bbox",
            }
        ]
    }

    data = json.dumps(todisplay)
    data = base64.urlsafe_b64encode(json.dumps(todisplay).encode())

    url = 'http://immersivepoints.com/oculus.html?jsonb64=' + data.decode('ascii')
    print(url)
    iframe = '<iframe src=' + url + ' width=1000 height=550></iframe>'
    IPython.display.HTML(iframe)
    


http://immersivepoints.com/oculus.html?jsonb64=eyJwb2ludHMiOiBbeyJzb3VyY2UiOiAidXJsIiwgInVybCI6ICJodHRwczovL3BvaW50Y2xvdWR2aXN1YWxpemF0aW9uLnMzLmV1LWNlbnRyYWwtMS5hbWF6b25hd3MuY29tL2FzdHl4L2FzdHl4MDAwMDAwLnR4dC54eXpyZ2IiLCAidHlwZSI6ICJYWVpSR0IifV0sICJiYm94ZXMiOiBbeyJzb3VyY2UiOiAidXJsIiwgInVybCI6ICJodHRwczovL3BvaW50Y2xvdWR2aXN1YWxpemF0aW9uLnMzLmV1LWNlbnRyYWwtMS5hbWF6b25hd3MuY29tL2FzdHl4L2FzdHl4YmJveDAwMDAwMC50eHQuYmJveCJ9XX0=
http://immersivepoints.com/oculus.html?jsonb64=eyJwb2ludHMiOiBbeyJzb3VyY2UiOiAidXJsIiwgInVybCI6ICJodHRwczovL3BvaW50Y2xvdWR2aXN1YWxpemF0aW9uLnMzLmV1LWNlbnRyYWwtMS5hbWF6b25hd3MuY29tL2FzdHl4L2FzdHl4MDAwMDAxLnR4dC54eXpyZ2IiLCAidHlwZSI6ICJYWVpSR0IifV0sICJiYm94ZXMiOiBbeyJzb3VyY2UiOiAidXJsIiwgInVybCI6ICJodHRwczovL3BvaW50Y2xvdWR2aXN1YWxpemF0aW9uLnMzLmV1LWNlbnRyYWwtMS5hbWF6b25hd3MuY29tL2FzdHl4L2FzdHl4YmJveDAwMDAwMS50eHQuYmJveCJ9XX0=
http://immersivepoints.com/oculus.html?jsonb64=eyJwb2ludHMiOiBbeyJzb3VyY2UiOiAidXJsIiwgInVybCI6ICJodHRwczovL3BvaW50Y2xvdWR2aXN1YWxpemF0aW9uLnMzL